In [0]:
"""
Simple Customer Support Agent using Vector Search MCP Server
Run this in a Databricks notebook after your Vector Search MCP is set up
"""

from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksMCPClient
import asyncio

# Your MCP server URL
VECTOR_SEARCH_MCP = "https://adb-395997151148170.10.azuredatabricks.net/api/2.0/mcp/vector-search/prod/customer_support"

# =============================================================================
# SIMPLE SYNCHRONOUS VERSION (Easy to understand)
# =============================================================================

def simple_support_agent(customer_issue: str):
    """
    Simple agent that searches for similar past tickets
    """
    from databricks.vector_search.client import VectorSearchClient
    
    # Initialize client
    vsc = VectorSearchClient(disable_notice=True)
    
    # Get the index
    index = vsc.get_index(
        endpoint_name="customer_support_endpoint",
        index_name="prod.customer_support.ticket_embeddings"
    )
    
    # Search for similar tickets
    results = index.similarity_search(
        query_text=customer_issue,
        columns=["ticket_id", "issue", "resolution", "satisfaction_score"],
        num_results=3
    )
    
    # Format response
    print(f"\n{'='*80}")
    print(f"CUSTOMER ISSUE: {customer_issue}")
    print(f"{'='*80}\n")
    
    print("🔍 Found similar past tickets:\n")
    
    if 'result' in results and 'data_array' in results['result']:
        for i, result in enumerate(results['result']['data_array'], 1):
            ticket_id = result[0]
            issue = result[1]
            resolution = result[2]
            satisfaction = result[3]
            
            print(f"{i}. Ticket {ticket_id} (⭐ {satisfaction}/5)")
            print(f"   Issue: {issue}")
            print(f"   Resolution: {resolution}\n")
    else:
        print("No similar tickets found.")
    
    print(f"{'='*80}\n")

# =============================================================================
# TEST THE SIMPLE AGENT
# =============================================================================

# Example 1: Network issue
simple_support_agent("My internet is very slow during evening hours")

# Example 2: Technical issue
simple_support_agent("Can't connect to 5G on my new phone")

# Example 3: Billing issue
simple_support_agent("I have questions about my bill")


# =============================================================================
# ADVANCED: Using MCP Client (Proper way for production)
# =============================================================================

async def advanced_support_agent(customer_issue: str):
    """
    Advanced agent using MCP client
    This is the proper way to use MCP servers
    """
    from databricks_mcp.oauth_provider import DatabricksOAuthClientProvider
    from mcp.client.streamable_http import streamablehttp_client
    from mcp.client.session import ClientSession
    from mcp.types import CallToolRequest
    
    # Initialize workspace client
    workspace_client = WorkspaceClient()
    
    print(f"\n{'='*80}")
    print(f"CUSTOMER ISSUE: {customer_issue}")
    print(f"{'='*80}\n")
    
    # Connect to MCP server
    async with streamablehttp_client(
        url=VECTOR_SEARCH_MCP,
        auth=DatabricksOAuthClientProvider(workspace_client)
    ) as (read_stream, write_stream, _):
        
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            
            # List available tools
            tools_response = await session.list_tools()
            print(f"📋 Available MCP tools: {[t.name for t in tools_response.tools]}\n")
            
            # Call the vector search tool
            result = await session.call_tool(
                name="ticket_embeddings",
                arguments={
                    "query": customer_issue,
                    "num_results": 3
                }
            )
            
            print("🔍 MCP Tool Response:")
            print(result.content[0].text)
    
    print(f"\n{'='*80}\n")

# Test advanced agent
# asyncio.run(advanced_support_agent("My internet is very slow"))


# =============================================================================
# FULL AGENT WITH LLM (Production-ready)
# =============================================================================

def full_support_agent(customer_id: str, customer_issue: str):
    """
    Complete agent that:
    1. Searches for similar tickets (Vector Search MCP)
    2. Gets customer info (from tables)
    3. Recommends actions (using an LLM)
    """
    from databricks.vector_search.client import VectorSearchClient
    
    print(f"\n{'='*80}")
    print(f"CUSTOMER SUPPORT REQUEST")
    print(f"{'='*80}")
    print(f"Customer ID: {customer_id}")
    print(f"Issue: {customer_issue}\n")
    
    # Step 1: Get customer info
    print("📊 Step 1: Retrieving customer information...")
    customer_info = spark.sql(f"""
        SELECT * FROM prod.customer_support.customers
        WHERE customer_id = '{customer_id}'
    """).collect()
    
    if customer_info:
        customer = customer_info[0].asDict()
        print(f"✓ Customer: {customer.get('name', 'Unknown')}")
        print(f"  Plan: {customer.get('plan', 'Unknown')}")
        print(f"  Data Usage: {customer.get('data_usage_gb', 0)} GB\n")
    else:
        print("⚠️ Customer not found\n")
        customer = {}
    
    # Step 2: Search for similar tickets using Vector Search MCP
    print("🔍 Step 2: Searching for similar past tickets...")
    vsc = VectorSearchClient(disable_notice=True)
    index = vsc.get_index(
        endpoint_name="customer_support_endpoint",
        index_name="prod.customer_support.ticket_embeddings"
    )
    
    results = index.similarity_search(
        query_text=customer_issue,
        columns=["ticket_id", "issue", "resolution", "satisfaction_score"],
        num_results=3
    )
    
    if 'result' in results and 'data_array' in results['result']:
        print(f"✓ Found {len(results['result']['data_array'])} similar tickets:\n")
        for i, result in enumerate(results['result']['data_array'], 1):
            print(f"  {i}. {result[0]}: {result[1]}")
            print(f"     Resolution: {result[2]} (⭐ {result[3]}/5)\n")
    
    # Step 3: Generate recommendations
    print("💡 Step 3: Recommended Actions:")
    if results.get('result', {}).get('data_array'):
        best_match = results['result']['data_array'][0]
        print(f"  1. Apply proven solution from {best_match[0]}")
        print(f"     {best_match[2]}")
        print(f"  2. Follow up in 24 hours to confirm resolution")
        print(f"  3. Document the outcome for future reference")
    
    print(f"\n{'='*80}\n")

# =============================================================================
# USAGE EXAMPLES
# =============================================================================

print("\n" + "="*80)
print("TESTING CUSTOMER SUPPORT AGENT")
print("="*80 + "\n")

# First, create some test customer data if it doesn't exist
try:
    spark.sql("""
    CREATE TABLE IF NOT EXISTS prod.customer_support.customers (
      customer_id STRING,
      name STRING,
      plan STRING,
      data_usage_gb DOUBLE
    ) USING DELTA
    """)
    
    spark.sql("""
    INSERT OVERWRITE prod.customer_support.customers VALUES
      ('CUST001', 'John Smith', 'Premium Unlimited', 45.2),
      ('CUST002', 'Sarah Johnson', 'Basic 5GB', 4.8),
      ('CUST003', 'Mike Davis', 'Premium Unlimited', 78.5)
    """)
    print("✓ Test customer data ready\n")
except:
    pass

# Test scenarios
print("\n📞 SCENARIO 1: Network Performance Issue")
print("-" * 80)
full_support_agent("CUST003", "My internet is extremely slow during peak hours")

print("\n📞 SCENARIO 2: Technical Support Issue")
print("-" * 80)
full_support_agent("CUST002", "I can't connect to 5G network on my new device")

print("\n📞 SCENARIO 3: General Network Issue")
print("-" * 80)
full_support_agent("CUST001", "Internet keeps disconnecting")

print("\n" + "="*80)
print("✅ ALL TESTS COMPLETE!")
print("="*80)

In [0]:
select * from  nadb